In [1]:
#!pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # plotting library
import json
#from sklearn.datasets.samples_generator import make_blobs
#from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

!pip install git+https://github.com/python-visualization/folium
import folium # map rendering library

import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


print("Modules imported.")
print("Hello Capstone Project Course!")


     |████████████████████████████████| 102kB 2.3MB/s ta 0:00:011
  Stored in directory: /home/nbuser/.cache/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib
You should consider upgrading via the 'pip install --upgrade pip' command.
  Cloning https://github.com/python-visualization/folium to /tmp/pip-req-build-g0wwkvxg
  Running command git clone -q https://github.com/python-visualization/folium /tmp/pip-req-build-g0wwkvxg
  Stored in directory: /tmp/pip-ephem-wheel-cache-f319cuo7/wheels/1e/e1/75/ecbc91fd5dd5d90befb0b533bf7492d38acffa033310731862
Successfully built folium
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.9.0+2.g5d14833 which is incompatible.
  Found existing installation: folium 0.2.1
    Uninstalling folium-0.2.1:
      Successfully uninstalled folium-0.2.1
You should consider upgrading via the 'pip install --upgrade pip' command.
Modules imported.
Hello Capstone Project Course!


### Download New York dataset 
##### This dataset is provided by Coursera that includes borough, neighborhood, postal code, latitude and longitude in a JSON format.

In [4]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [5]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)    
neighborhoods_data = newyork_data['features']
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

### Define a dataframe with necessary column names.
##### Create a table with Borough, Neighborhood, Latitude and Longitude and then populate the new york dataset into this table.

In [6]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']
df_newyork = pd.DataFrame(columns = column_names)
df_newyork

,Borough,Neighborhood,Latitude,Longitude


In [7]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    df_newyork = df_newyork.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [8]:
print("Size of New York dataframe: ",df_newyork.shape)
df_newyork.head()

Size of New York dataframe:  (306, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


*We have now populated a table necessary to plot these on the map.*

### Option 1: Extract Brooklyn Into A DF:

In [9]:
print(df_newyork["Borough"].unique())
df_brooklyn = df_newyork[df_newyork["Borough"] == "Brooklyn"].reset_index(drop=True)
df_brooklyn.head()

['Bronx' 'Manhattan' 'Brooklyn' 'Queens' 'Staten Island']


,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


### Option 2: Extract Brooklyn Into A DF:

In [10]:
brooklyn = df_newyork["Borough"] == "Brooklyn"
df_newyork[brooklyn].reset_index(drop=True).head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


# Plot  Map
*Plot a map with all postcodes and a current location red dot.*

In [11]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Brooklyn, NY are 40.6501038, -73.9495823.


In [250]:
# create map of Brooklyn using latitude and longitude values
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=12)

# add a red circle marker to represent the Brooklyn Center
folium.CircleMarker(
    [latitude, longitude],
    radius=13,
    color='red',
    popup='Brooklyn',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_brooklyn)


# add markers to map
for lat, lng, label in zip(df_brooklyn['Latitude'], df_brooklyn['Longitude'], df_brooklyn['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_brooklyn)     
map_brooklyn

After a decent map plotting all neighborhoods in Brooklyn, we want to show a list of all neighborhoods.

In [13]:
df_brooklyn["Neighborhood"].unique()

array(['Bay Ridge', 'Bensonhurst', 'Sunset Park', 'Greenpoint',
       'Gravesend', 'Brighton Beach', 'Sheepshead Bay',
       'Manhattan Terrace', 'Flatbush', 'Crown Heights', 'East Flatbush',
       'Kensington', 'Windsor Terrace', 'Prospect Heights', 'Brownsville',
       'Williamsburg', 'Bushwick', 'Bedford Stuyvesant',
       'Brooklyn Heights', 'Cobble Hill', 'Carroll Gardens', 'Red Hook',
       'Gowanus', 'Fort Greene', 'Park Slope', 'Cypress Hills',
       'East New York', 'Starrett City', 'Canarsie', 'Flatlands',
       'Mill Island', 'Manhattan Beach', 'Coney Island', 'Bath Beach',
       'Borough Park', 'Dyker Heights', 'Gerritsen Beach', 'Marine Park',
       'Clinton Hill', 'Sea Gate', 'Downtown', 'Boerum Hill',
       'Prospect Lefferts Gardens', 'Ocean Hill', 'City Line',
       'Bergen Beach', 'Midwood', 'Prospect Park South', 'Georgetown',
       'East Williamsburg', 'North Side', 'South Side', 'Ocean Parkway',
       'Fort Hamilton', 'Ditmas Park', 'Wingate', 'Rugby'

In [251]:
# Use the first row as an example to test Foursquare API connection.

neighborhood_latitude = df_brooklyn.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_brooklyn.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_brooklyn.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bay Ridge are 40.625801065010656, -74.03062069353813.


### Connect to Foursquare API
##### Investigate the get request and filter out relevant dataset that we want to use for the clustering.

In [253]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=1,2,3,4'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
#url # display URL - Commented to mask foursquare credentials


##### Display JSON results

In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cdfd8086a6071493e9218f3'},
 'response': {'headerLocation': 'Bay Ridge',
  'headerFullLocation': 'Bay Ridge, Brooklyn',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 53,
  'suggestedBounds': {'ne': {'lat': 40.63030106951066,
    'lng': -74.02470273356597},
   'sw': {'lat': 40.62130106051065, 'lng': -74.03653865351028}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad09cf7f964a520bed820e3',
       'name': 'Bagel Boy',
       'location': {'address': '8002 3rd Ave',
        'crossStreet': '80th St',
        'lat': 40.627896,
        'lng': -74.029335,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.627896,
          'lng': -74.029335}],
        'distance': 257,
        'postalCode': '11209',
   

In [18]:
df_temp0 =json_normalize(
    results['response']['groups'][0]['items']
)

df_temp0.columns.values

array(['reasons.count', 'reasons.items', 'referralId', 'venue.categories',
       'venue.delivery.id', 'venue.delivery.provider.icon.name',
       'venue.delivery.provider.icon.prefix',
       'venue.delivery.provider.icon.sizes',
       'venue.delivery.provider.name', 'venue.delivery.url', 'venue.id',
       'venue.location.address', 'venue.location.cc',
       'venue.location.city', 'venue.location.country',
       'venue.location.crossStreet', 'venue.location.distance',
       'venue.location.formattedAddress', 'venue.location.labeledLatLngs',
       'venue.location.lat', 'venue.location.lng',
       'venue.location.postalCode', 'venue.location.state', 'venue.name',
       'venue.photos.count', 'venue.photos.groups', 'venue.venuePage.id'],
      dtype=object)

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']
    
#normalize to flatten json into a table structure    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.id', 'venue.name', 'venue.categories', 'venue.location.distance', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row from the function created above
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,id,name,categories,distance,lat,lng
0,4ad09cf7f964a520bed820e3,Bagel Boy,Bagel Shop,257,40.627896,-74.029335
1,538ca7d5498ec684c6387a46,Cocoa Grinder,Juice Bar,205,40.623967,-74.030863
2,4abe4e84f964a520958c20e3,Pegasus Cafe,Breakfast Spot,296,40.623168,-74.031186
3,4a343f1ef964a520f69b1fe3,Leo's Casa Calamari,Pizza Place,275,40.623348,-74.031082
4,4ea17d24e5facb29c6d81d6f,Ho' Brah Taco Joint,Taco Place,322,40.622960,-74.031371


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

53 venues were returned by Foursquare.


### Explore Venue
##### We need to explore venues to retrieve specific statistics.

In [254]:
# function that extracts the category of the venue
def get_price(row):
    try:
        categories_list = row['price']['tier']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['tier']

In [256]:
#We will use one of the examples above to fetch the rating
#55e47d26498e44d12c7c004e
#501036cae4b0616139c2759d
venue_id_temp = '4abe4e84f964a520958c20e3' 
url_temp = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}&price=1,2,3,4'.format(venue_id_temp, CLIENT_ID, CLIENT_SECRET, VERSION)
url_temp
df_tempresult0 = requests.get(url_temp).json()['response']['venue']
df_tempresult0

KeyError: 'venue'

In [128]:
df_tempresult0 = requests.get(url_temp).json()['response']['venue']
abc = json_normalize(df_tempresult0)
abc.columns.values


array(['allowMenuUrlEdit', 'attributes.groups', 'beenHere.count',
       'beenHere.lastCheckinExpiredAt', 'beenHere.marked',
       'beenHere.unconfirmedCount', 'bestPhoto.createdAt',
       'bestPhoto.height', 'bestPhoto.id', 'bestPhoto.prefix',
       'bestPhoto.source.name', 'bestPhoto.source.url',
       'bestPhoto.suffix', 'bestPhoto.visibility', 'bestPhoto.width',
       'canonicalUrl', 'categories', 'colors.algoVersion',
       'colors.highlightColor.photoId', 'colors.highlightColor.value',
       'colors.highlightTextColor.photoId',
       'colors.highlightTextColor.value', 'contact.facebook',
       'contact.facebookName', 'contact.facebookUsername',
       'contact.formattedPhone', 'contact.phone', 'createdAt',
       'description', 'dislike', 'hereNow.count', 'hereNow.groups',
       'hereNow.summary', 'hours.dayData', 'hours.isLocalHoliday',
       'hours.isOpen', 'hours.richStatus.entities',
       'hours.richStatus.text', 'hours.status', 'hours.timeframes', 'id',
       '

In [129]:
try:
    print(df_tempresult0['price']['tier'])
except:
    print(df_tempresult0["attributes"]["groups"][0]["items"][0]["priceTier"])

3


In [255]:
columns_att_ha = ["name", "categories", 'location.lat', 'location.lng', 
                  'rating', 'ratingSignals','likes.count', 
                  'dislike', 'price.tier']
tips_filtered_temp = abc.loc[:, columns_att_ha]

# filter the category for each row from the function created above
tips_filtered_temp['categories'] = tips_filtered_temp.apply(get_category_type, axis=1)
tips_filtered_temp.columns = [col.split(".")[-1] for col in tips_filtered_temp.columns]

tips_filtered_temp

,name,categories,lat,lng,rating,ratingSignals,count,dislike,tier
0,Le Garage,French Restaurant,40.699312,-73.926591,8.2,115,88,False,3


## Extract all venues for each neighborhood

In [66]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=1,2,3,4'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue ID',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [257]:
df_brooklynVenues = getNearbyVenues(names=df_brooklyn['Neighborhood'],
                                   latitudes=df_brooklyn['Latitude'],
                                   longitudes=df_brooklyn['Longitude']
                                  )

print("Size of Brooklyn venues dataframe: ", df_brooklynVenues.shape)
df_brooklynVenues.head()

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus
Size of Brooklyn venues dataframe:  (1055, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,4ad09cf7f964a520bed820e3,Bagel Boy,40.627896,-74.029335,Bagel Shop
1,Bay Ridge,40.625801,-74.030621,538ca7d5498ec684c6387a46,Cocoa Grinder,40.623967,-74.030863,Juice Bar
2,Bay Ridge,40.625801,-74.030621,5553702c498ecb85f99d8f37,XIN,40.625082,-74.030494,Chinese Restaurant
3,Bay Ridge,40.625801,-74.030621,4b58f8f2f964a5203f7628e3,Peppino's,40.629851,-74.028481,Pizza Place
4,Bay Ridge,40.625801,-74.030621,4b70d8dff964a52063312de3,Tuscany Grill,40.622913,-74.031387,Italian Restaurant


## Extract all venue details. 
##### Retreving venue price tier and number of likes. 

In [294]:
def getNearbyVenuesDetails(venue_id, venue_name, venue_lat, venue_lng):
    
    venues_list=[]
    for venue_id, name, lat, lng in zip(venue_id, venue_name, venue_lat, venue_lng):
        print(venue_id, name)
    
    # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION
            )
        #print(url) #Commented to mask api credentials. 
    
    # make the get request
        results_temp = requests.get(url).json()["response"]["venue"]
        results = []
        results.append(results_temp)
        
            
    # return only relevant information for each nearby venue
        venues_list.append([(
            neighborhood_name, 
            neighborhood_latitude, 
            neighborhood_longitude,
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],
            v['price']['tier'],
            v['likes']['count'],
            v['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
              'Neighborhood Latitude', 
              'Neighborhood Longitude',
              'Venue', 
              'Venue Latitude', 
              'Venue Longitude',
              'Venue Price Class',
              'Venue Likes',
              'Venue Category']
    return nearby_venues


brooklyn_venueDetails = getNearbyVenuesDetails(venue_id=nearby_venues['id'],
                venue_name=nearby_venues['name'],
                venue_lat=nearby_venues['lat'],
                venue_lng=nearby_venues['lng'])




4ad09cf7f964a520bed820e3 Bagel Boy


KeyError: 'venue'

In [260]:
#Sort the dataframe by venue category
brooklyn_venueDetails.sort_values("Venue Category")

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Price Class,Venue Likes,Venue Category
50,Bay Ridge,40.625801,-74.030621,Cappuccino Cafe,40.629510,-74.028539,2,3,American Restaurant
21,Bay Ridge,40.625801,-74.030621,Chadwick's Restaurant,40.621450,-74.031964,3,49,American Restaurant
18,Bay Ridge,40.625801,-74.030621,Greenhouse Cafe,40.629542,-74.028514,3,45,American Restaurant
0,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,1,89,Bagel Shop
40,Bay Ridge,40.625801,-74.030621,Fantastic Bagels,40.622420,-74.028576,1,6,Bagel Shop
31,Bay Ridge,40.625801,-74.030621,H & L Bagels,40.621625,-74.031831,1,16,Bagel Shop
45,Bay Ridge,40.625801,-74.030621,Harp Bar Brooklyn,40.629789,-74.028533,2,13,Bar
44,Bay Ridge,40.625801,-74.030621,The Pour House of Bay Ridge,40.628517,-74.028963,2,22,Bar
8,Bay Ridge,40.625801,-74.030621,The Kettle Black,40.622839,-74.031411,2,117,Bar
2,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,2,78,Breakfast Spot


In [261]:
df_category_count = brooklyn_venueDetails.groupby("Venue Category").count()[["Neighborhood"]].rename(columns={"Neighborhood": "Venue Count"})
df_category_count.head()

,Venue Count
Venue Category,
American Restaurant,3
Bagel Shop,3
Bar,3
Breakfast Spot,1
Café,1


### Transform table to be used for K-Means Clustering
##### Calculate the mean price class and venue likes then do a left join to the above df_category_count.

In [262]:
df_categorytable = brooklyn_venueDetails[["Venue Category", "Venue Price Class", "Venue Likes"]].groupby("Venue Category").mean().join(df_category_count, on=["Venue Category"], how="left").reset_index()

In [263]:
df_categorytable.head()

,Venue Category,Venue Price Class,Venue Likes,Venue Count
0,American Restaurant,2.666667,32.333333,3
1,Bagel Shop,1.000000,37.000000,3
2,Bar,2.000000,50.666667,3
3,Breakfast Spot,2.000000,78.000000,1
4,Café,1.000000,6.000000,1


In [264]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venueDetails[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venueDetails['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]


In [265]:
brooklyn_onehot.head()

,Neighborhood,American Restaurant,Bagel Shop,Bar,Breakfast Spot,Café,Chinese Restaurant,Coffee Shop,Dim Sum Restaurant,Diner,...,Pizza Place,Sandwich Place,Seafood Restaurant,Snack Place,Sports Bar,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Vietnamese Restaurant
0,Bay Ridge,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bay Ridge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bay Ridge,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bay Ridge,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,Bay Ridge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [266]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped

,Neighborhood,American Restaurant,Bagel Shop,Bar,Breakfast Spot,Café,Chinese Restaurant,Coffee Shop,Dim Sum Restaurant,Diner,...,Pizza Place,Sandwich Place,Seafood Restaurant,Snack Place,Sports Bar,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Vietnamese Restaurant
0,Bay Ridge,0.056604,0.056604,0.056604,0.018868,0.018868,0.037736,0.018868,0.018868,0.018868,...,0.075472,0.037736,0.037736,0.018868,0.018868,0.037736,0.018868,0.018868,0.037736,0.018868


In [267]:
num_top_venues = 5

for hood in brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = brooklyn_grouped[brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bay Ridge----
                 venue  freq
0   Italian Restaurant  0.09
1          Pizza Place  0.08
2     Greek Restaurant  0.06
3           Bagel Shop  0.06
4  American Restaurant  0.06




In [268]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [269]:
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Bay Ridge,Italian Restaurant,Pizza Place,Bagel Shop,Bar,Greek Restaurant,American Restaurant,Hookah Bar


In [270]:
df_categorytable

,Venue Category,Venue Price Class,Venue Likes,Venue Count
0,American Restaurant,2.666667,32.333333,3
1,Bagel Shop,1.000000,37.000000,3
2,Bar,2.000000,50.666667,3
3,Breakfast Spot,2.000000,78.000000,1
4,Café,1.000000,6.000000,1
5,Chinese Restaurant,2.500000,12.500000,2
6,Coffee Shop,1.000000,48.000000,1
7,Dim Sum Restaurant,2.000000,12.000000,1
8,Diner,1.000000,4.000000,1
9,Donut Shop,1.000000,25.000000,1


In [271]:
# set number of clusters
kclusters = 3

brooklyn_grouped_clustering = df_categorytable.drop('Venue Category', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 2, 0, 0, 2, 0, 0, 0], dtype=int32)

In [272]:
# add clustering labels
df_categorytable.insert(0, 'Cluster Labels', kmeans.labels_)

# join categorytable with brooklyn_venuDetails to add latitude/longitude for each venue
df_categories_merged = df_categorytable.drop(["Venue Price Class", "Venue Likes", "Venue Count"],1).set_index("Venue Category").join(brooklyn_venueDetails.set_index("Venue Category"), on="Venue Category", how="left")

df_categories_merged.head()

,Cluster Labels,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Price Class,Venue Likes
Venue Category,,,,,,,,,
American Restaurant,0,Bay Ridge,40.625801,-74.030621,Greenhouse Cafe,40.629542,-74.028514,3,45
American Restaurant,0,Bay Ridge,40.625801,-74.030621,Chadwick's Restaurant,40.621450,-74.031964,3,49
American Restaurant,0,Bay Ridge,40.625801,-74.030621,Cappuccino Cafe,40.629510,-74.028539,2,3
Bagel Shop,0,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,1,89
Bagel Shop,0,Bay Ridge,40.625801,-74.030621,H & L Bagels,40.621625,-74.031831,1,16


In [247]:
df_categories_merged.columns.values

array(['Cluster Labels', 'Neighborhood', 'Neighborhood Latitude',
       'Neighborhood Longitude', 'Venue', 'Venue Latitude',
       'Venue Longitude', 'Venue Price Class', 'Venue Likes'],
      dtype=object)

### Venues Low Popularity
##### Check where Cluster Labels = 0

In [273]:
cluster_0 = df_categories_merged["Cluster Labels"] == 0
df_categories_merged[cluster_0]

,Cluster Labels,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Price Class,Venue Likes
Venue Category,,,,,,,,,
American Restaurant,0,Bay Ridge,40.625801,-74.030621,Greenhouse Cafe,40.629542,-74.028514,3,45
American Restaurant,0,Bay Ridge,40.625801,-74.030621,Chadwick's Restaurant,40.621450,-74.031964,3,49
American Restaurant,0,Bay Ridge,40.625801,-74.030621,Cappuccino Cafe,40.629510,-74.028539,2,3
Bagel Shop,0,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,1,89
Bagel Shop,0,Bay Ridge,40.625801,-74.030621,H & L Bagels,40.621625,-74.031831,1,16
Bagel Shop,0,Bay Ridge,40.625801,-74.030621,Fantastic Bagels,40.622420,-74.028576,1,6
Café,0,Bay Ridge,40.625801,-74.030621,Coffee Rx,40.622115,-74.031908,1,6
Chinese Restaurant,0,Bay Ridge,40.625801,-74.030621,XIN,40.625082,-74.030494,3,6
Chinese Restaurant,0,Bay Ridge,40.625801,-74.030621,86 Noodles,40.622573,-74.028423,2,19


### Venues High Popularity
##### Check where Cluster Labels = 1

In [274]:
cluster_1 = df_categories_merged["Cluster Labels"] == 1
df_categories_merged[cluster_1]

,Cluster Labels,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Price Class,Venue Likes
Venue Category,,,,,,,,,
Taco Place,1,Bay Ridge,40.625801,-74.030621,Ho' Brah Taco Joint,40.62296,-74.031371,2,208


### Venues Neutral Popularity 
##### Check where Cluster Labels = 2

In [276]:
cluster_2 = df_categories_merged["Cluster Labels"] == 2
df_categories_merged[cluster_2]

,Cluster Labels,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Price Class,Venue Likes
Venue Category,,,,,,,,,
Bar,2,Bay Ridge,40.625801,-74.030621,The Kettle Black,40.622839,-74.031411,2,117
Bar,2,Bay Ridge,40.625801,-74.030621,The Pour House of Bay Ridge,40.628517,-74.028963,2,22
Bar,2,Bay Ridge,40.625801,-74.030621,Harp Bar Brooklyn,40.629789,-74.028533,2,13
Breakfast Spot,2,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,2,78
Coffee Shop,2,Bay Ridge,40.625801,-74.030621,Mocha Mocha Cafe,40.622699,-74.028636,1,48
Juice Bar,2,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,1,42
Middle Eastern Restaurant,2,Bay Ridge,40.625801,-74.030621,Karam,40.622931,-74.028316,2,66
New American Restaurant,2,Bay Ridge,40.625801,-74.030621,Cebu' Bar & Bistro,40.621812,-74.031727,3,119
Sports Bar,2,Bay Ridge,40.625801,-74.030621,Windy City Ale House,40.628117,-74.029128,2,58


In [293]:
# create map
map_clusters = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=16)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add a red circle marker to represent the Bay Ridge Center
folium.CircleMarker(
    [neighborhood_latitude, neighborhood_longitude],
    radius=13,
    color='yellow',
    popup='Bay Ridge Center',
    fill = True,
    fill_color = 'yellow',
    fill_opacity = 0.6
).add_to(map_clusters)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_categories_merged['Venue Latitude'], df_categories_merged['Venue Longitude'], df_categories_merged['Venue'], df_categories_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters